In [1]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
sec = SEC()
fin = ADatabase("dividends")
umod = UniversalModeler()

In [3]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [4]:
factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity',
"commonstockdividendspersharecashpaid"]

In [5]:
included_columns=[
    "year",
    "quarter",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [7]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        filing = sec.retrieve_filing_data(cik)
        filing = p.column_date_processing(filing)
        filing = filing.groupby(["year","quarter"]).mean().reset_index()
        filing["commonstockdividendspersharecashpaid"] = filing["commonstockdividendspersharecashpaid"].fillna(0)
        ticker_data = prices.copy()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data = ticker_data.merge(filing,on=["year","quarter"],how="left").reset_index()
        ticker_data["y"] = ticker_data["adjclose"].shift(-4) + ticker_data["commonstockdividendspersharecashpaid"]
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

  1%|█                                                                                                                                                                          | 3/490 [00:00<02:18,  3.51it/s]

'commonstockdividendspersharecashpaid'


  1%|█▋                                                                                                                                                                         | 5/490 [00:01<01:56,  4.16it/s]

'commonstockdividendspersharecashpaid'


  1%|██                                                                                                                                                                         | 6/490 [00:01<01:56,  4.15it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 488/490 [02:56<00:00,  2.99it/s]

'commonstockdividendspersharecashpaid'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [02:57<00:00,  1.88it/s]

"['cashandcashequivalentsatcarryingvalue'] not in index"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [02:57<00:00,  2.75it/s]


In [8]:
data = pd.concat(training_sets)

In [9]:
data = data.dropna()

In [10]:
data

,year,quarter,ticker,adjclose,y,assets,liabilitiesandstockholdersequity,incometaxexpensebenefit,retainedearningsaccumulateddeficit,accumulatedothercomprehensiveincomelossnetoftax,earningspersharebasic,earningspersharediluted,propertyplantandequipmentnet,cashandcashequivalentsatcarryingvalue,entitycommonstocksharesoutstanding,weightedaveragenumberofdilutedsharesoutstanding,weightedaveragenumberofsharesoutstandingbasic,stockholdersequity,commonstockdividendspersharecashpaid
48,2012,1,MMM,62.705929,77.472781,2.967400e+10,3.088600e+10,1.551333e+09,2.717150e+10,-4.284000e+09,2.555455,2.519091,7.472500e+09,2.621250e+09,694543763.0,7.170667e+08,7.075667e+08,1.554150e+10,2.113333
49,2012,2,MMM,62.787037,81.307970,3.181550e+10,3.181550e+10,4.520000e+08,2.860300e+10,-4.905000e+09,1.565000,1.540000,7.710500e+09,2.504800e+09,693872048.0,7.162500e+08,7.041500e+08,1.580100e+10,0.570000
50,2012,3,MMM,66.710493,87.814722,3.247750e+10,3.247750e+10,6.990000e+08,2.890650e+10,-5.018500e+09,2.440000,2.400000,7.686000e+09,3.070000e+09,691319463.0,7.149750e+08,7.039250e+08,1.591300e+10,0.855000
51,2012,4,MMM,67.131673,96.917189,3.304750e+10,3.304750e+10,9.145000e+08,2.924900e+10,-4.786500e+09,3.217500,3.172500,7.802500e+09,3.000250e+09,691931278.0,7.113250e+08,7.015750e+08,1.653200e+10,1.140000
52,2013,1,MMM,75.359447,102.721556,3.188267e+10,3.274600e+10,1.702000e+09,2.951350e+10,-4.887500e+09,2.783636,2.743636,8.022000e+09,2.879750e+09,689990255.0,7.159333e+08,7.053667e+08,1.649750e+10,2.220000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2020,3,ZTS,150.821274,199.153900,1.230700e+10,1.230700e+10,1.307500e+08,4.737500e+09,-7.910000e+08,1.167500,1.160000,1.975500e+09,2.643500e+09,475144704.0,4.804250e+08,4.771750e+08,2.845000e+09,0.000000
31,2020,4,ZTS,159.798699,216.397228,1.263500e+10,1.263500e+10,1.897500e+08,4.976500e+09,-7.305000e+08,1.735000,1.720000,2.019500e+09,2.794000e+09,475267819.0,4.802000e+08,4.768750e+08,3.155000e+09,0.000000
33,2021,2,ZTS,171.169916,171.429767,1.370250e+10,1.370250e+10,1.015000e+08,5.879000e+09,-6.815000e+08,1.035000,1.025000,2.214500e+09,3.603000e+09,474766701.0,4.784500e+08,4.755500e+08,3.929000e+09,0.000000
34,2021,3,ZTS,199.153900,165.970366,1.383900e+10,1.383900e+10,1.662500e+08,6.075500e+09,-6.765000e+08,1.450000,1.442500,2.244000e+09,3.631000e+09,473944193.0,4.778000e+08,4.751750e+08,4.059500e+09,0.000000


In [ ]:
fin.connect()
years = 4
fin.drop("sim")
for modeler in [umod]:
    for year in tqdm(range(2016,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors,tf=False)
        stuff["training_years"] = years
        fin.store(set_name,stuff)
fin.disconnect()

 43%|██████████████████████████████████████████████████████████████████████████▏                                                                                                  | 3/7 [00:28<00:37,  9.47s/it]

In [ ]:
stuff